In [ ]:
# Run it on google colab
!uv pip install --system -r pyproject.toml

# Run it on local notebook
# !uv sync

Resolved 190 packages in 19ms
⠙ Preparing packages... (0/3)                                                   
⠙ Preparing packages... (0/3)-------------------     0 B/105.74 KiB          
⠙ Preparing packages... (0/3)-------------------     0 B/105.74 KiB          
click                ------------------------------     0 B/105.74 KiB
⠙ Preparing packages... (0/3)-------------------     0 B/110.63 KiB          
click                ------------------------------ 16.00 KiB/105.74 KiB
⠙ Preparing packages... (0/3)-------------------     0 B/110.63 KiB          
click                ------------------------------ 16.00 KiB/105.74 KiB
⠙ Preparing packages... (0/3)------------------- 14.88 KiB/110.63 KiB        
click                ------------------------------ 32.00 KiB/105.74 KiB
⠙ Preparing packages... (0/3)------------------- 14.88 KiB/110.63 KiB        
click                ------------------------------ 48.00 KiB/105.74 KiB
⠙ Preparing packages... (0/3)------------------- 14.88 KiB

In [1]:
# Setup: Dependencies, Env Vars, Config Flags
import os
import json
import re
import logging
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, Tuple

import numpy as np
import pandas as pd
import yfinance as yf
import ta

from dotenv import load_dotenv

# Load env early so TensorFlow honors GPU flags
load_dotenv()
USE_GPU = os.environ.get("USE_GPU", "false").lower() == "true"
if not USE_GPU:
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model

import torch
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

# Logging
LOG_FILE = Path("agent_run_log.txt").resolve()
logging.basicConfig(filename=str(LOG_FILE), level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s", force=True)

# OpenRouter / LLM
OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY")
OPENROUTER_BASE_URL = os.environ.get("OPENROUTER_BASE_URL", "https://openrouter.ai/api/v1")
MODEL_NAME = os.environ.get("OPENROUTER_MODEL", "deepseek/deepseek-chat-v3.1")
LLM_TEMPERATURE = float(os.environ.get("LLM_TEMPERATURE", "0.3"))
MAX_TOKENS = int(os.environ.get("LLM_MAX_TOKENS", "800"))

# Alpaca
ALPACA_API_KEY = os.environ.get("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.environ.get("ALPACA_SECRET_KEY")
ALPACA_BASE_URL = os.environ.get("ALPACA_BASE_URL", "https://paper-api.alpaca.markets")

# General flags
ENABLE_SERVER = os.environ.get("ENABLE_SERVER", "false").lower() == "true"
DRY_RUN = os.environ.get("DRY_RUN", "true").lower() == "true"
DEFAULT_TICKER = os.environ.get("DEFAULT_TICKER", "AAPL")
DEFAULT_PERIOD = os.environ.get("DEFAULT_PERIOD", "5y")
LOOKBACK_DAYS = int(os.environ.get("LOOKBACK_DAYS", "60"))
TRADE_QUANTITY = int(os.environ.get("TRADE_QUANTITY", "1"))

# Model paths
LSTM_MODEL_PATH = Path("models/lstm_model.keras").resolve()
SCALER_JSON_PATH = Path("models/scaler.json").resolve()
COMBINER_WEIGHTS_PATH = Path("models/combiner_weights_90day.pt").resolve()

2025-12-16 21:32:46.714198: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-16 21:32:47.870610: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-16 21:33:38.636526: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
# Data Utils: fetch_stock_data, compute_indicators
def fetch_stock_data(ticker: str, period: str = DEFAULT_PERIOD) -> pd.DataFrame:
    data = yf.download(tickers=ticker, period=period, interval="1d", auto_adjust=False, progress=False)
    if data.empty:
        raise ValueError(f"No data retrieved for ticker {ticker} and period {period}.")
    if isinstance(data.columns, pd.MultiIndex):
        if ticker in data.columns.get_level_values(-1):
            data = data.xs(ticker, axis=1, level=-1)
        else:
            data.columns = data.columns.get_level_values(0)
    for col in ["Open", "High", "Low", "Close", "Adj Close", "Volume"]:
        if col in data.columns and isinstance(data[col], pd.DataFrame):
            data[col] = data[col].squeeze("columns")
    data = data.dropna(how="all")
    data.index = pd.to_datetime(data.index)
    return data

def compute_indicators(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        raise ValueError("Input price DataFrame is empty.")
    data = df.copy()
    close = data["Close"]
    high = data["High"]
    low = data["Low"]
    volume = data["Volume"]
    data["SMA_10"] = ta.trend.SMAIndicator(close=close, window=10).sma_indicator()
    data["SMA_20"] = ta.trend.SMAIndicator(close=close, window=20).sma_indicator()
    data["SMA_50"] = ta.trend.SMAIndicator(close=close, window=50).sma_indicator()
    data["SMA_100"] = ta.trend.SMAIndicator(close=close, window=100).sma_indicator()
    data["SMA_200"] = ta.trend.SMAIndicator(close=close, window=200).sma_indicator()
    data["EMA_12"] = ta.trend.EMAIndicator(close=close, window=12).ema_indicator()
    data["EMA_26"] = ta.trend.EMAIndicator(close=close, window=26).ema_indicator()
    data["RSI_14"] = ta.momentum.RSIIndicator(close=close, window=14).rsi()
    macd = ta.trend.MACD(close=close)
    data["MACD"] = macd.macd()
    data["MACD_Signal"] = macd.macd_signal()
    data["MACD_Hist"] = macd.macd_diff()
    bb = ta.volatility.BollingerBands(close=close, window=20, window_dev=2)
    data["BB_Upper"] = bb.bollinger_hband()
    data["BB_Middle"] = bb.bollinger_mavg()
    data["BB_Lower"] = bb.bollinger_lband()
    stoch = ta.momentum.StochasticOscillator(high=high, low=low, close=close, window=14, smooth_window=3)
    data["Stoch_K"] = stoch.stoch()
    data["Stoch_D"] = stoch.stoch_signal()
    obv = ta.volume.OnBalanceVolumeIndicator(close=close, volume=volume)
    data["OBV"] = obv.on_balance_volume()
    data["Daily_Return"] = close.pct_change()
    data = data.dropna()
    return data

In [4]:
# LSTM: Load Model + Scaler, Predict, Signal Vector
def load_scaler_from_json(path: Path) -> MinMaxScaler:
    scaler = MinMaxScaler()
    with open(path, 'r') as f:
        params = json.load(f)
    if 'min_' in params: scaler.min_ = np.array(params['min_'])
    if 'scale_' in params: scaler.scale_ = np.array(params['scale_'])
    if 'data_min_' in params: scaler.data_min_ = np.array(params['data_min_'])
    if 'data_max_' in params: scaler.data_max_ = np.array(params['data_max_'])
    if 'data_range_' in params: scaler.data_range_ = np.array(params['data_range_'])
    if 'n_features_in_' in params: scaler.n_features_in_ = params['n_features_in']
    return scaler

def lstm_predict_next_close(ticker: str = DEFAULT_TICKER) -> Tuple[float, float]:
    model = load_model(str(LSTM_MODEL_PATH))
    scaler = load_scaler_from_json(SCALER_JSON_PATH)
    data = yf.download(ticker, period="3y", interval="1d")[['Close']].dropna()
    scaled = scaler.transform(data)
    last_100 = scaled[-100:].reshape(1, 100, 1)
    pred_scaled = model.predict(last_100)
    pred_next_close = scaler.inverse_transform(pred_scaled)[0][0]
    last_close = float(data['Close'].iloc[-1])
    return float(pred_next_close), last_close

def lstm_decision_vector(ticker: str = DEFAULT_TICKER, threshold: float = 0.005) -> Tuple[np.ndarray, str]:
    pred_next, last_close = lstm_predict_next_close(ticker)
    change = (pred_next - last_close) / last_close
    # Simple soft mapping
    buy = float(max(0.0, change))
    sell = float(max(0.0, -change))
    hold = float(max(0.0, 1 - abs(change) * 200))  # heuristic
    vec = np.array([buy, sell, hold], dtype=float)
    s = vec.sum()
    vec = vec / s if s > 0 else np.array([0.33, 0.33, 0.34])
    label = 'BUY' if change > threshold else ('SELL' if change < -threshold else 'HOLD')
    return vec, label

In [5]:
# LLM: Fetch Data, Indicators, Prompt, Parse Decision Vector
from functools import lru_cache

@lru_cache(maxsize=1)
def get_llm_client() -> ChatOpenAI:
    if not OPENROUTER_API_KEY:
        raise EnvironmentError("OPENROUTER_API_KEY is not set")
    return ChatOpenAI(
        model=MODEL_NAME,
        openai_api_key=OPENROUTER_API_KEY,
        openai_api_base=OPENROUTER_BASE_URL,
        temperature=LLM_TEMPERATURE,
        max_tokens=MAX_TOKENS,
        timeout=90,
    )

def build_llm_input(df: pd.DataFrame, lookback_days: int = LOOKBACK_DAYS) -> str:
    window = df.tail(lookback_days)
    display_cols = ["Close","SMA_10","SMA_50","SMA_200","EMA_12","EMA_26","RSI_14","MACD","MACD_Signal","MACD_Hist","BB_Upper","BB_Middle","BB_Lower","Stoch_K","Stoch_D","OBV","Volume","Daily_Return"]
    available_cols = [c for c in display_cols if c in window.columns]
    snapshot = window[available_cols].round(4)
    lines = []
    for idx, row in snapshot.iterrows():
        lines.append(
            f"{idx.date().isoformat()} | "
            + f"Close={float(row.get('Close', float('nan'))):.2f}, "
            + f"RSI14={float(row.get('RSI_14', float('nan'))):.1f}, "
            + f"MACD={float(row.get('MACD', float('nan'))):.3f}, "
            + f"Signal={float(row.get('MACD_Signal', float('nan'))):.3f}, "
            + f"BBU={float(row.get('BB_Upper', float('nan'))):.2f}, BBL={float(row.get('BB_Lower', float('nan'))):.2f}, "
            + f"StochK={float(row.get('Stoch_K', float('nan'))):.1f}, Vol={float(row.get('Volume', float('nan'))):.0f}"
        )
    price_change_pct = (window["Close"].iloc[-1] / window["Close"].iloc[0] - 1) * 100
    daily_returns = window["Daily_Return"].dropna()
    annual_vol = daily_returns.std() * np.sqrt(252) * 100 if not daily_returns.empty else 0.0
    avg_volume = window["Volume"].mean() if "Volume" in window.columns else float("nan")
    summary = dedent(f"""Recent performance summary:\n- Days: {len(window)}\n- Net change: {price_change_pct:.2f}%\n- Ann vol: {annual_vol:.2f}%\n- Avg vol: {avg_volume:,.0f}""").strip()
    return summary + "\n\n" + "\n".join(lines)

def build_llm_prompt(ticker: str, market_snapshot: str, lookback_days: int = LOOKBACK_DAYS) -> str:
    return dedent(f"""You are a disciplined trading assistant. Evaluate ticker {ticker}.\n\nData is daily; predict next day's close direction. Return strict JSON with keys: buy_confidence, sell_confidence, hold_confidence, next_day_view, explanation. Confidences in [0,1] summing ~1. next_day_view in [BUY, SELL, HOLD]. No extra text.\n\nSnapshot (last {lookback_days} days):\n{market_snapshot}""").strip()

def _strip_json_fences(text: str) -> str:
    cleaned = text.strip()
    if cleaned.startswith("```"):
        cleaned = re.sub(r"^```(?:json)?", "", cleaned, flags=re.IGNORECASE).strip()
        cleaned = re.sub(r"```$", "", cleaned).strip()
    return cleaned

def parse_llm_decision(raw_text: str) -> Dict[str, Any]:
    cleaned = _strip_json_fences(raw_text)
    try:
        payload = json.loads(cleaned)
    except json.JSONDecodeError:
        m = re.search(r"\{.*\}", cleaned, flags=re.DOTALL)
        if not m:
            raise ValueError("LLM response not valid JSON")
        payload = json.loads(m.group())
    req = {"buy_confidence","sell_confidence","hold_confidence","next_day_view","explanation"}
    missing = req - payload.keys()
    if missing:
        raise ValueError(f"LLM response missing keys: {sorted(missing)}")
    conf = {}
    for k in ["buy_confidence","sell_confidence","hold_confidence"]:
        v = float(payload[k])
        conf[k] = max(0.0, min(1.0, v))
    total = sum(conf.values())
    if total <= 0:
        raise ValueError("Confidence sum is zero")
    conf = {k: v/total for k,v in conf.items()}
    ndv = str(payload.get("next_day_view","HOLD")).upper().strip()
    if ndv not in {"BUY","SELL","HOLD"}: ndv = "HOLD"
    return {"buy_confidence": conf["buy_confidence"], "sell_confidence": conf["sell_confidence"], "hold_confidence": conf["hold_confidence"], "next_day_view": ndv, "explanation": str(payload.get("explanation",""))}

def llm_decision_vector(ticker: str = DEFAULT_TICKER, period: str = DEFAULT_PERIOD) -> Tuple[np.ndarray, Dict[str,Any]]:
    price = fetch_stock_data(ticker, period)
    enriched = compute_indicators(price)
    snapshot = build_llm_input(enriched, lookback_days=LOOKBACK_DAYS)
    prompt = build_llm_prompt(ticker, snapshot)
    client = get_llm_client()
    messages = [SystemMessage(content="Respond with strict JSON only."), HumanMessage(content=prompt)]
    response = client.invoke(messages)
    payload = parse_llm_decision(response.content)
    logging.info(json.dumps({"ticker": ticker, "payload": payload}))
    vec = np.array([payload["buy_confidence"], payload["sell_confidence"], payload["hold_confidence"]], dtype=float)
    return vec, payload

In [6]:
# Combiner: Load Weights, Combine Vectors, Final Decision
def load_combiner_weights(path: Path) -> Tuple[float, float]:
    try:
        weights = torch.load(str(path), weights_only=True)
    except TypeError:
        weights = torch.load(str(path))
    alpha = float(weights.get('alpha', 0.5))
    beta = float(weights.get('beta', 0.5))
    return alpha, beta


def combine_vectors(llm_vec: np.ndarray, lstm_vec: np.ndarray, alpha: float, beta: float) -> Tuple[np.ndarray, str]:
    combined = alpha * llm_vec + beta * lstm_vec
    labels = ['BUY', 'SELL', 'HOLD']
    label = labels[int(np.argmax(combined))]
    return combined, label

In [7]:
# Alpaca Trading: Client, Risk Checks, Place Order
import requests

def _normalized_alpaca_base() -> str:
    base = (ALPACA_BASE_URL or "https://paper-api.alpaca.markets").strip()
    base = base.rstrip("/")
    if base.lower().endswith("/v2"):
        base = base[:-3].rstrip("/")
    return base or "https://paper-api.alpaca.markets"


def _alpaca_endpoint(path: str) -> str:
    base = _normalized_alpaca_base()
    return f"{base}/{path.lstrip('/')}"


def _alpaca_headers() -> Dict[str, str]:
    return {
        "APCA-API-KEY-ID": ALPACA_API_KEY or "",
        "APCA-API-SECRET-KEY": ALPACA_SECRET_KEY or "",
        "Content-Type": "application/json",
    }


def place_order_alpaca(symbol: str, side: str, qty: int = TRADE_QUANTITY, time_in_force: str = "day") -> Dict[str, Any]:
    qty = int(TRADE_QUANTITY)
    if side not in {"buy","sell"}:
        return {"error": "invalid side"}
    if DRY_RUN:
        return {"status": "DRY_RUN", "symbol": symbol, "side": side, "qty": qty}
    if not (ALPACA_API_KEY and ALPACA_SECRET_KEY):
        return {"error": "Missing Alpaca API keys"}
    url = _alpaca_endpoint("v2/orders")
    payload = {
        "symbol": symbol,
        "qty": qty,
        "side": side,
        "type": "market",
        "time_in_force": time_in_force,
    }
    resp = requests.post(url, headers=_alpaca_headers(), json=payload, timeout=30)
    try:
        return resp.json()
    except Exception:
        return {"status": resp.status_code, "text": resp.text}

In [8]:
# Orchestration: Run-One Decision + Optional Trade
def run_agent(ticker: str = DEFAULT_TICKER) -> Dict[str, Any]:
    lstm_vec, lstm_label = lstm_decision_vector(ticker)
    llm_vec, llm_payload = llm_decision_vector(ticker)
    alpha, beta = load_combiner_weights(COMBINER_WEIGHTS_PATH)
    combined_vec, combined_label = combine_vectors(llm_vec, lstm_vec, alpha, beta)
    trade_result = None
    if combined_label != 'HOLD':
        side = 'buy' if combined_label == 'BUY' else 'sell'
        trade_result = place_order_alpaca(symbol=ticker, side=side)
    result = {
        "ticker": ticker,
        "lstm": {"vector": lstm_vec.tolist(), "label": lstm_label},
        "llm": {"vector": llm_vec.tolist(), "view": llm_payload.get("next_day_view"), "explanation": llm_payload.get("explanation")},
        "combiner": {"alpha": alpha, "beta": beta, "vector": combined_vec.tolist(), "label": combined_label},
        "trade": trade_result,
    }
    print(json.dumps(result, indent=2))
    logging.info(json.dumps(result))
    return result

# Single-run entrypoint
_ = run_agent(DEFAULT_TICKER)

2025-12-16 12:07:04.827390: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/tmp/ipykernel_5385/2100403759.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="3y", interval="1d")[['Close']].dropna()
/tmp/ipykernel_5385/2100403759.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="3y", interval="1d")[['Close']].dropna()
[*********************100%***********************]  1 of 1 completed



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 922ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 922ms/step


/tmp/ipykernel_5385/2100403759.py:22: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  last_close = float(data['Close'].iloc[-1])


{
  "ticker": "AAPL",
  "lstm": {
    "vector": [
      0.0,
      1.0,
      0.0
    ],
    "label": "SELL"
  },
  "llm": {
    "vector": [
      0.35,
      0.45,
      0.2
    ],
    "view": "SELL",
    "explanation": "RSI at 49.8 indicates neutral but declining from overbought levels, MACD below signal line suggests weakening momentum, recent price near lower Bollinger Band but volume drop to 20.2M signals lack of buying interest, following a downtrend from recent highs with increased volatility."
  },
  "combiner": {
    "alpha": 0.2093292474746704,
    "beta": 1.333786129951477,
    "vector": [
      0.07326523661613464,
      1.4279842913150786,
      0.04186584949493408
    ],
    "label": "SELL"
  },
  "trade": {
    "status": "DRY_RUN",
    "symbol": "AAPL",
    "side": "sell",
    "qty": 1
  }
}


In [ ]:
# VS Code: Integrated Tests and Commands
DRY_RUN = True # Override for tests

def _test_parse_llm_decision() -> None:
    sample = json.dumps({
        "buy_confidence": 0.6,
        "sell_confidence": 0.2,
        "hold_confidence": 0.2,
        "next_day_view": "BUY",
        "explanation": "Momentum remains bullish.",
    })
    parsed = parse_llm_decision(sample)
    assert abs(parsed["buy_confidence"] - 0.6) < 1e-6
    assert parsed["next_day_view"] == "BUY"


def _test_combine_vectors() -> None:
    llm_vec = np.array([0.7, 0.1, 0.2])
    lstm_vec = np.array([0.4, 0.4, 0.2])
    combined, label = combine_vectors(llm_vec, lstm_vec, alpha=0.2, beta=1.0)
    assert combined.shape == (3,)
    assert label == "BUY"


def _fetch_alpaca_order(order_id: str) -> Dict[str, Any]:
    if not order_id:
        raise AssertionError("order_id is required")
    if not (ALPACA_API_KEY and ALPACA_SECRET_KEY):
        raise AssertionError("Missing Alpaca API credentials")
    url = _alpaca_endpoint(f"v2/orders/{order_id}")
    resp = requests.get(url, headers=_alpaca_headers(), timeout=30)
    try:
        return resp.json()
    except Exception:
        return {"status": resp.status_code, "text": resp.text}


def _test_alpaca_place_buy_with_sample_vectors(symbol: str = DEFAULT_TICKER) -> None:
    if DRY_RUN:
        raise AssertionError("Set DRY_RUN=false in the environment to run live Alpaca trade tests.")
    llm_vec = np.array([0.75, 0.15, 0.10])
    lstm_vec = np.array([0.55, 0.25, 0.20])
    combined, label = combine_vectors(llm_vec, lstm_vec, alpha=0.3, beta=0.9)
    assert label == "BUY", "Sample vectors must produce a BUY signal"
    order = place_order_alpaca(symbol=symbol, side="buy")
    order_id = order.get("id") or order.get("order_id") or order.get("client_order_id")
    assert order_id, f"Unexpected Alpaca response: {order}"
    status_payload = _fetch_alpaca_order(order_id)
    status = status_payload.get("status", "").lower()
    allowed = {"new", "accepted", "partially_filled", "filled", "done_for_day"}
    assert status in allowed, f"Order status not confirmed: {status_payload}"
    print(f"Live Alpaca trade confirmed with status '{status_payload.get('status')}' for order {order_id}")

def run_smoke_tests() -> None:
    _test_parse_llm_decision()
    _test_combine_vectors()
    if not DRY_RUN:
        _test_alpaca_place_buy_with_sample_vectors()
    else:
        print("Skipping live Alpaca trade test because DRY_RUN=true.")
    print("Smoke tests passed.")

run_smoke_tests()

# VS Code terminal helpers:
#   uv sync
#   uv run jupyter nbconvert --to notebook --execute auto_stock_trader_agent.ipynb

Live Alpaca trade confirmed with status 'new' for order f3f573f3-4d05-48b2-a407-641398cbff47
Smoke tests passed.


In [ ]:
# FastAPI Server: Flag-Controlled Startup
ENABLE_SERVER = True # Override for tests

import asyncio
from fastapi import FastAPI
from fastapi.responses import JSONResponse
import uvicorn

app = FastAPI()


@app.get("/health")
def health():
    return {"status": "ok"}


@app.get("/decision")
def get_decision(ticker: str = DEFAULT_TICKER):
    res = run_agent(ticker)
    return JSONResponse(res)


@app.post("/trade")
def trigger_trade(ticker: str = DEFAULT_TICKER):
    res = run_agent(ticker)
    return JSONResponse(res)


async def _serve_fastapi() -> None:
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    await server.serve()

if ENABLE_SERVER:
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        asyncio.run(_serve_fastapi())
    else:
        loop.create_task(_serve_fastapi())
        print("FastAPI server running at http://0.0.0.0:8000")

FastAPI server running at http://0.0.0.0:8000


INFO:     Started server process [5385]
INFO:     Waiting for application startup.
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
/tmp/ipykernel_5385/2100403759.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="3y", interval="1d")[['Close']].dropna()
/tmp/ipykernel_5385/2100403759.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="3y", interval="1d")[['Close']].dropna()
[*********************100%***********************]  1 of 1 completed



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step


/tmp/ipykernel_5385/2100403759.py:22: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  last_close = float(data['Close'].iloc[-1])
/tmp/ipykernel_5385/2100403759.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="3y", interval="1d")[['Close']].dropna()
/tmp/ipykernel_5385/2100403759.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="3y", interval="1d")[['Close']].dropna()
[*********************100%***********************]  1 of 1 completed



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step


...

{
  "ticker": "AAPL",
  "lstm": {
    "vector": [
      0.0,
      1.0,
      0.0
    ],
    "label": "SELL"
  },
  "llm": {
    "vector": [
      0.25,
      0.4,
      0.35
    ],
    "view": "SELL",
    "explanation": "RSI near 50 indicates neutral momentum, but recent price decline from 286.19 to 274.86 suggests bearish pressure. MACD declining and below signal line, StochK low at 18.2, and price near lower Bollinger Band support. High volatility and recent downtrend support cautious sell bias."
  },
  "combiner": {
    "alpha": 0.2093292474746704,
    "beta": 1.333786129951477,
    "vector": [
      0.0523323118686676,
      1.4175178289413453,
      0.07326523661613464
    ],
    "label": "SELL"
  },
  "trade": {
    "status": "DRY_RUN",
    "symbol": "AAPL",
    "side": "sell",
    "qty": 1
  }
}
INFO:     127.0.0.1:51050 - "GET /decision?ticker=AAPL HTTP/1.1" 200 OK


/tmp/ipykernel_5385/2100403759.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="3y", interval="1d")[['Close']].dropna()
[*********************100%***********************]  1 of 1 completed



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step


/tmp/ipykernel_5385/2100403759.py:22: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  last_close = float(data['Close'].iloc[-1])


{
  "ticker": "AAPL",
  "lstm": {
    "vector": [
      0.0,
      1.0,
      0.0
    ],
    "label": "SELL"
  },
  "llm": {
    "vector": [
      0.35,
      0.25,
      0.4
    ],
    "view": "HOLD",
    "explanation": "RSI near 51 indicates neutral momentum, MACD declining but above signal, recent volatility and mixed signals suggest consolidation; no strong directional bias."
  },
  "combiner": {
    "alpha": 0.2093292474746704,
    "beta": 1.333786129951477,
    "vector": [
      0.07326523661613464,
      1.3861184418201447,
      0.08373169898986817
    ],
    "label": "SELL"
  },
  "trade": {
    "status": "DRY_RUN",
    "symbol": "AAPL",
    "side": "sell",
    "qty": 1
  }
}
INFO:     127.0.0.1:48160 - "GET /decision?ticker=AAPL HTTP/1.1" 200 OK
